In [7]:
import sys
!{sys.executable} -m pip install -q yfinance pandas numpy

In [8]:
import yfinance as yf
import pandas as pd
import numpy as np
from math import erf, sqrt, log, exp


In [9]:
ticker = 'SPY'
t = yf.Ticker(ticker)
expiries = t.options
expiries[:10], len(expiries)

(('2026-01-22',
  '2026-01-23',
  '2026-01-26',
  '2026-01-27',
  '2026-01-28',
  '2026-01-29',
  '2026-01-30',
  '2026-02-06',
  '2026-02-13',
  '2026-02-20'),
 29)

In [10]:
expiry = expiries[0]
chain = t.option_chain(expiry)
calls = chain.calls
puts =  chain.puts

calls.head(), puts.head()

(       contractSymbol             lastTradeDate  strike  lastPrice  bid  ask  \
 0  SPY260122C00500000 2026-01-21 18:32:00+00:00   500.0     180.75  0.0  0.0   
 1  SPY260122C00615000 2026-01-21 16:39:05+00:00   615.0      67.23  0.0  0.0   
 2  SPY260122C00633000 2026-01-21 17:44:46+00:00   633.0      47.29  0.0  0.0   
 3  SPY260122C00640000 2026-01-20 14:42:03+00:00   640.0      42.41  0.0  0.0   
 4  SPY260122C00641000 2026-01-21 19:13:39+00:00   641.0      39.38  0.0  0.0   
 
    change  percentChange  volume  openInterest  impliedVolatility  inTheMoney  \
 0     0.0            0.0    15.0             0            0.00001        True   
 1     0.0            0.0    58.0             0            0.00001        True   
 2     0.0            0.0     8.0             0            0.00001        True   
 3     0.0            0.0     2.0             0            0.00001        True   
 4     0.0            0.0     8.0             0            0.00001        True   
 
   contractSize cu

In [11]:

def clean_chain(df, expiry, option_type):
    df = df.copy()

    # Keep only columns that exist (yfinance uses lastPrice and openInterest)
    keep = ["strike", "impliedVolatility", "lastPrice", "bid", "ask", "volume", "openInterest"]
    df = df[[c for c in keep if c in df.columns]]

    df["expiry"] = expiry
    df["type"] = option_type  # "call" or "put"

    # Mid price (fallback to lastPrice if bid/ask missing)
    if "bid" in df.columns and "ask" in df.columns:
        b = df["bid"].fillna(0.0)
        a = df["ask"].fillna(0.0)
        mid = (b + a) / 2.0
        df["mid"] = np.where((b > 0) & (a > 0), mid, df.get("lastPrice", 0.0))
    else:
        df["mid"] = df.get("lastPrice", 0.0)

    # Clean IV
    if "impliedVolatility" in df.columns:
        df = df[df["impliedVolatility"].notna()]
        df = df[(df["impliedVolatility"] > 0) & (df["impliedVolatility"] < 5)]

    return df

In [12]:
ticker = yf.Ticker("SPY")

expiry = ticker.options[0]
chain = ticker.option_chain(expiry)

calls = clean_chain(chain.calls, expiry, 'call')
puts = clean_chain(chain.puts, expiry, 'put')

df = pd.concat([calls, puts], ignore_index=True)

df.head()



,strike,impliedVolatility,lastPrice,bid,ask,volume,openInterest,expiry,type,mid
0,500.0,0.00001,180.75,0.0,0.0,15.0,0.0,2026-01-22,call,180.75
1,615.0,0.00001,67.23,0.0,0.0,58.0,0.0,2026-01-22,call,67.23
2,633.0,0.00001,47.29,0.0,0.0,8.0,0.0,2026-01-22,call,47.29
3,640.0,0.00001,42.41,0.0,0.0,2.0,0.0,2026-01-22,call,42.41
4,641.0,0.00001,39.38,0.0,0.0,8.0,0.0,2026-01-22,call,39.38


In [14]:
from datetime import datetime

VAL_DATE = pd.Timestamp('today').normalize()
df['val_date'] = VAL_DATE

df['expiry_dt'] = pd.to_datetime(df['expiry'])
df['T'] = (df['expiry_dt'] - df["val_date"]).dt.days / 365.0

df = df[df['T'] > 0]
df[["strike", "type", "impliedVolatility", "T"]].head()

,strike,type,impliedVolatility,T


In [15]:
required = {"strike", "expiry", "type"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}. Have: {list(df.columns)}")



In [16]:
if "bid" in df.columns and "ask" in df.columns:
    df['price'] = (df['bid'] + df['ask'] / 2.0)
elif 'lastPrice' in df.columns:
    df["price"] = df['lastPrice']
else: raise ValueError("No usable price column found (bid/ask or lastPrice).")


In [17]:
# Basic quote sanity checks
if "bid" in df.columns and "ask" in df.columns:
    df = df[(df["bid"].fillna(0) >= 0) & (df["ask"].fillna(0) >= 0)]
    df = df[(df["ask"].fillna(0) == 0) | (df["ask"] >= df["bid"])]  # allow zeros but enforce ask>=bid

# Price must be positive
df = df[df["price"].notna() & (df["price"] > 0)]

# Implied vol sanity range (adjust if needed)
if "impliedVolatility" in df.columns:
    df = df[df["impliedVolatility"].notna()]
    df = df[(df["impliedVolatility"] > 0.0001) & (df["impliedVolatility"] < 5.0)]

In [ ]:
# Spot price (from market data or assumed for now)
SPOT = 100
SPOT = spot

# Simple moneyness
df["moneyness"] = df["strike"] / SPOT

# Log-Moneyness 
df["log_moneyness"] = np.log(df['strike'] / SPOT)

df[["strike", "moneyness", "log_moneyness"]].head()



NameError: name 'spot' is not defined

In [ ]:
# Drop bad or unusable rows
df_surf = df_surf.dropna(subset=["T", "impliedVolatility"])

# Bounds of inputs
df_surf = df_surf[(df_surf['T'] > 0) & (df_surf["T"] < 5.0)]
df_surf = df_surf[(df_surf["impliedVolatility"] > 0.01) & (df_surf["impliedVolatilty"] < 5.0)]




In [ ]:
# Use spot (or forward) to compute log-moneyness
SPOT = spot

df_surf['logM'] = np.log(df_surf["strike"] / SPOT)

# Bucket T into tenors to make a grid
tenor_bins = [0, 1/12, 3/12, 6/12, 1, 2, 5]
tenor_labels = ["1M", "3M", "6M", "1Y", "2Y", "5Y"]

df_surf["tenor"] =  pd.concat(df_surf['T'], bins=tenor_bins, labels=tenor_labels, right=True)

df_surf[['expiry', 'T', 'tenor', 'strike', 'logM', 'impliedVolatility', 'type']].head()

In [ ]:
import sys
print(sys.executable)

from lawson_quant_library.util import Calendar
from lawson_quant_library.parameter import IRCurve, DivCurve, EQVol
from lawson_quant_library.instrument import EQOption

REFERENCE_DATE = "2026-01-06"
cal = Calendar("US:NYSE")
spot = 100.0

/Users/lawsonprendergast/miniforge3/envs/lawson-quant/bin/python


In [ ]:
ir_curve = IRCurve.from_deposit_quotes(
    {"1M": 0.0500, "3M": 0.0520, "6M": 0.0530, "1Y": 0.0550},
    reference_date=REFERENCE_DATE,
    name="USD_IR",
)

div_curve = DivCurve(0.00)

In [ ]:
tenors = ["1M", "3M", "6M", "1Y"]
strikes = [80, 90, 100, 110, 120]

# vols[tenor_index][strike_index]
vol_grid = [
    [0.28, 0.24, 0.20, 0.21, 0.23],  # 1M
    [0.26, 0.22, 0.19, 0.20, 0.22],  # 3M
    [0.25, 0.21, 0.18, 0.19, 0.21],  # 6M
    [0.24, 0.20, 0.17, 0.18, 0.20],  # 1Y
]

vol = EQVol(currency="USD")
vol.set_surface_vol(
    strikes=strikes,
    tenors=tenors,
    vols=vol_grid,
    reference_date=REFERENCE_DATE,
)

TypeError: in method 'new_BlackVarianceSurface', argument 3 of type 'std::vector< Date,std::allocator< Date > > const &'

In [ ]:
prices = {}
for t in tenors:
    maturity = cal.add_tenor(REFERENCE_DATE, t)
    prices[t] = {}
    for k in strikes:
        opt = EQOption(
            spot=spot,
            strike=float(k),
            maturity_date=maturity,
            option_type="call",
            ir_curve=ir_curve,
            div_curve=div_curve,
            vol=vol,
            pricing_engine="bs_analytic",
        )
        prices[t][k] = opt.price()

prices

{'1M': {80: 20.344061636814953,
  90: 10.54676728320051,
  100: 2.537574195318202,
  110: 0.19279588414611862,
  120: 0.008933988968604032},
 '3M': {80: 21.164747451057345,
  90: 11.872247463940335,
  100: 4.406031317421481,
  110: 1.1764007536359113,
  120: 0.3089280199203187},
 '6M': {80: 22.597561367152956,
  90: 13.723213254508327,
  100: 6.372229002044891,
  110: 2.6582524848978952,
  120: 1.1903974715023746},
 '1Y': {80: 25.45321986795321,
  90: 16.925743560003717,
  100: 9.526075964610488,
  110: 5.386235569178359,
  120: 3.3378861432671827}}

In [ ]:
greeks = {}
for t in tenors:
    maturity = cal.add_tenor(REFERENCE_DATE, t)
    greeks[t] = {}
    for k in strikes:
        opt = EQOption(
            spot=spot,
            strike=float(k),
            maturity_date=maturity,
            option_type="call",
            ir_curve=ir_curve,
            div_curve=div_curve,
            vol=vol,
            pricing_engine="bs_analytic",
        )
        greeks[t][k] = {"delta": opt.delta(), "vega": opt.vega()}

# Example: ATM greeks at 3M
greeks["3M"][100.0]

{'delta': 0.5722179651480831, 'vega': 19.48454299383948}

In [ ]:
def call(k, tenor):
    return EQOption(
        spot=spot,
        strike=float(k),
        maturity_date=cal.add_tenor(REFERENCE_DATE, tenor),
        option_type="call",
        ir_curve=ir_curve,
        div_curve=div_curve,
        vol=vol,
        pricing_engine="bs_analytic",
    )

def put(k, tenor):
    return EQOption(
        spot=spot,
        strike=float(k),
        maturity_date=cal.add_tenor(REFERENCE_DATE, tenor),
        option_type="put",
        ir_curve=ir_curve,
        div_curve=div_curve,
        vol=vol,
        pricing_engine="bs_analytic",
    )

tenor = "3M"
atm = 100.0

straddle_price = call(atm, tenor).price() + put(atm, tenor).price()
rr_proxy_price = call(110.0, tenor).price() - put(90.0, tenor).price()

straddle_price, rr_proxy_price

(7.5475961273242325, 0.44217314646243466)

In [ ]:
tenor = "6M"
strike_sweep = [70, 80, 90, 100, 110, 120, 130]

sweep_prices = []
for k in strike_sweep:
    sweep_prices.append(call(k, tenor).price())

list(zip(strike_sweep, sweep_prices))

[(70, 31.968873139989075),
 (80, 22.597561367152956),
 (90, 13.723213254508327),
 (100, 6.372229002044891),
 (110, 2.6582524848978952),
 (120, 1.1903974715023746),
 (130, 0.5649424570557939)]